## The [Monty Hall](https://en.wikipedia.org/wiki/Monty_Hall_problem) problem:

Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a car; behind the others, goats. You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, "Do you want to pick door No. 2?" Is it to your advantage to switch your choice?

We know that if you always switch that means the only time you lose is if it happened that the car was under the door you originally picked which has a 33% chance of happening. Meaning you win 2/3 of the time by choosing to switch instead of staying.

Now this is generally considered an unintuitive problem. People may think the probability is 50% of picking the car given in the end you have a choice between 2 doors. In order to break this suspicion we will test it out using what is called a [Monte Carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method) simulation where you just put a bunch of random points and count how many of them give you a winning position given your decisions.

In [1]:
import numpy as np
import pandas as pd

In [2]:
num_trials = 10000
df_car = pd.DataFrame(np.random.randint(3, size=num_trials),columns=['car_under'])
df_picked = pd.DataFrame(np.random.randint(3, size=num_trials),columns=['picked'])

In order to figure out which is the best choice to make, we will simulate both decisions and see which has the highest probability of success. So we will place the car under a random door numbered from 0 to 2 (inclusive) and we will make a choice of a door and run the monty hall problem.

1. For the case where we do not switch we will simply check the equality of the 2 tables (== does an elementwise equality and returns a series of booleans), if they have the same value then we picked the door with the car under it. Now to get the expected value of our probability in this case we do $$ \text{Approximate }P(staying) = \frac{\text{Number of times we got the car}}{\text{Total number of trials}} $$
Which has the expected value as:
$$ E(staying) = \frac{1}{3} $$

In [3]:
num_times_we_got_the_car = (df_car['car_under'] == df_picked['picked']).value_counts()[True]
# value_counts() returns a series with the unique values as the index
# and the number of occurences of that value as the object\
approximate_P_staying = num_times_we_got_the_car / num_trials
approximate_P_staying

0.332

Now we want to get the: $$ \text{Approximate }P(switching) = \frac{\text{Number of times we got the car after we switched}}{\text{Total number of trials}}$$
With an expected value of:
$$ E(switching) = \frac{2}{3} $$

In [4]:
# lets begin by switching our choice
# (remember we always skip the door with the goat under it because the presenter opened it)
# so for example if we pick 0 and there's a goat in 1 we now pick 2
df_picked['car_under'] = df_car
# puts the 2 columns next to each other
def change_mind(row):
    picked = row[0]
    car_under = row[1]
    options = [0,1,2]
    options.remove(picked)
    options.remove(options[1] if options[0] == car_under else options[0])
    # we will remove the sheep that is under one of the other doors as an option
    return options[0]

# the change_mind function takes a row and switches doors based on the rules of the Monty Hall problem

df_changed_mind = df_picked.apply(change_mind,axis=1)

# now we do the same thing as before and figure out how many times we got this right
num_times_we_got_the_car = (df_car['car_under'] == df_changed_mind).value_counts()[True]
approximate_P_switching = num_times_we_got_the_car / num_trials
approximate_P_switching

0.668

As you can see the P(switching) is approximately 2/3 while the P(staying) is approximately 1/3. We also tried this for 10000 trials, so we are pretty confident that the result is close to the truth.